# Calculating a vector sum in PyOpenCL

Elwin van 't Wout

PUC Chile

25-9-2024

Calculate the sum of a vector with OpenCL.

First, we need to configure the virtual machine and install PyOpenCL.

In [1]:
!sudo apt update
!sudo apt install nvidia-cuda-toolkit -y
!pip install pyopencl

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [2]:
import numpy as np
import pyopencl as cl
import pyopencl.array as cl_array

/usr/local/lib/python3.10/dist-packages/pytools/persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


In [3]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [4]:
platform = cl.get_platforms()[0]
device = ctx.devices[0]
print("Platform name:", platform.name)
print("Device name:", device.name)
print("Maximum work group size:", device.max_work_group_size)

Platform name: NVIDIA CUDA
Device name: Tesla T4
Maximum work group size: 1024


In this tutorial, we like to calculate the sum of all elements in a vector of arbitrary size. In general, the size of the vector may not be a multiple of the desired workgroup size. In that case, the algorithm needs to be adapted to facilitate arbitrary workgroup and vector sizes. One option is called 'padding'.

Let us first create the kernel to calculate the sum of an integer array. See tutorial 5 for an explanation on the kernel.

In [5]:
kernel = """
__kernel void sum1(__global const long int *vec,
                   __global long int *partial_sums)
{
  int group_size = get_local_size(0);
  int local_id = get_local_id(0);
  int group_id = get_group_id(0);
  int global_id = get_global_id(0);

  if (local_id == 0){
    long int sum = 0;
    for(int i = 0; i < group_size; i++){
      sum += vec[global_id + i];
    }
    partial_sums[group_id] = sum;
  }
}
__kernel void sum2(__global long int *vec,
                   __global long int *partial_sums)
{
  int group_size = get_local_size(0);
  int local_id = get_local_id(0);
  int group_id = get_group_id(0);
  int global_id = get_global_id(0);

  int step = 2;
  while (step <= group_size) {
    if (local_id % step == 0) {
      vec[global_id] += vec[global_id + step / 2];
    }
    barrier(CLK_GLOBAL_MEM_FENCE);
    step *= 2;
  }
  if (local_id == 0){
    partial_sums[group_id] = vec[global_id];
  }
}
"""

In [6]:
prg = cl.Program(ctx, kernel).build()

The idea of 'padding' is to add dummy elements to the vector that will not change the final result. For example, if we'd like to calculate the sum of a vector, one can add an arbitrary number of elements with value zero, without changing the final result.

Let us assume that we have a vector of dimension $d$ and a workgroup of size $s$. PyOpenCL needs a domain with workgroups of equal size. However, $d$ may not be a multiple of $s$. Hence, we create another vector with size $n$ such that $n \geq d$ and $n \mod d = 0$, that is, $n$ is a multiple of $d$. The following function provides an efficient routine to calculate the next multiple.

In [7]:
def next_multiple(val, mul):
    """Return the smallest value which is larger or equal to 'val' and a multiple of 'mul'."""
    return -(-val // mul) * mul

For example, if we have a vector of size 100 and like to use workgroup sizes of 32, we need 4 groups to cover the dimension. Notice that $4 \cdot 32 = 128$ is the next multiple.

In [8]:
print("The next multiple of 32 larger or equal to 100 is: ", next_multiple(100, 32))

The next multiple of 32 larger or equal to 100 is:  128


Let us choose a vector size and a workgroup size.

In [9]:
vector_size = 10000
workgroup_size = 32

Let us calculate the next multiple of the workgroup size larger or equal to the vector dimension. This will be the size of our thread domain.

In [10]:
global_size = next_multiple(vector_size, workgroup_size)
n_workgroups = global_size // workgroup_size
print("The global size of the domain is:", global_size)
print("The number of workgroups is:", n_workgroups)

The global size of the domain is: 10016
The number of workgroups is: 313


Let us create a vector with $n$ values from zero to $n-1$ for which we'd like to calculate the sum of its elements.

In [11]:
my_vector = np.arange(vector_size, dtype=np.int64)

The essential step of the 'padding' approach is to create another vector which will be given to the PyOpenCL kernel. That is, we need to add additional zero elements to fill the vector up until reaching the desired size. Remember that appending zero elements will not change the objective: calculating the sum of the vector.

In [12]:
padding = np.zeros(global_size - vector_size, dtype=np.int64)
np_vector = np.concatenate((my_vector, padding))
print("The size of the padded vector:", np_vector.size)
print("The elements of the last workgroup:", np_vector[-workgroup_size:])

The size of the padded vector: 10016
The elements of the last workgroup: [9984 9985 9986 9987 9988 9989 9990 9991 9992 9993 9994 9995 9996 9997
 9998 9999    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [13]:
print("Sum of the original vector:", np.sum(my_vector))
print("Sum of the padded vector:", np.sum(np_vector))

Sum of the original vector: 49995000
Sum of the padded vector: 49995000


We can indeed see that the new vector has zero elements in the final workgroup. These are the padded values. Now, we are ready to launch the kernel for the padded vector. Notice that we need to provide the global size of the domain, not the dimension of the vector to the program.

In [14]:
cl_vector = cl_array.to_device(queue, np_vector)

cl_partial_sums1 = cl_array.empty(queue, n_workgroups, dtype=np.int64)
cl_partial_sums2 = cl_array.empty(queue, n_workgroups, dtype=np.int64)

In [15]:
event = prg.sum1(queue,
                 (global_size,),
                 (workgroup_size,),
                 cl_vector.data,
                 cl_partial_sums1.data
                )

In [16]:
event = prg.sum2(queue,
                 (global_size,),
                 (workgroup_size,),
                 cl_vector.data,
                 cl_partial_sums2.data
                )

In [17]:
np_partial_sums1 = cl_partial_sums1.get()
vector_sum1 = np.sum(np_partial_sums1)
np_partial_sums2 = cl_partial_sums2.get()
vector_sum2 = np.sum(np_partial_sums2)

In [18]:
print("The sum calculated by OpenCL:", vector_sum1)
print("The sum calculated by OpenCL:", vector_sum2)
print("The exact value of the sum:  ", vector_size*(vector_size-1)//2)

The sum calculated by OpenCL: 49995000
The sum calculated by OpenCL: 49995000
The exact value of the sum:   49995000


The exact value of summing values ranging from 0 to $n-1$ is explicitly known: $n(n-1)/2$. The results implemented with OpenCL are correct, indeed.